## Regret Analysis of OFL - \<Performance Experiments\>

In [5]:
import tensorflow as tf
import keras
from keras import layers, models

kernel_initializer = tf.keras.initializers.RandomNormal(stddev=0.01)
bias_initializer=tf.keras.initializers.Zeros()

# my_model = tf.keras.Sequential([
#             tf.keras.Input(shape=(14, 1)),
#             layers.Dense(64, activation='relu', kernel_initializer=kernel_initializer, bias_initializer=bias_initializer),
#             layers.Dense(64, activation='relu', kernel_initializer=kernel_initializer, bias_initializer=bias_initializer),
#             layers.Dense(1)
# ])

my_model = tf.keras.Sequential([
            tf.keras.Input(14, 1),
            layers.Dense(64, activation="relu"),
            layers.Dense(64, activation="relu"),
            layers.Dense(4, activation="softmax")
])

my_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (1, 64)                   960       
                                                                 
 dense_4 (Dense)             (1, 64)                   4160      
                                                                 
 dense_5 (Dense)             (1, 4)                    260       
                                                                 
Total params: 5,380
Trainable params: 5,380
Non-trainable params: 0
_________________________________________________________________


### 1. Environment Setting

In [2]:
import  torch
import  numpy as np
import  matplotlib.pyplot as plt
from    keras import datasets
import  import_ipynb

from RegretBound import opt_param
from data import MNIST_data, Air_data, Room_data, Cond_data, Tom_data, Energy_data
from model import OFL_Model
from utils import sigma_diff

K = 1000        # Number of clients
D = 34826       # number of parameters (MNIST)
D = 4353        # (Air, Cond, Tom)
#D = 5380         # (Room)
P = 0.01        # Com. overhead reduction rate from FedOGD
L = 8           # Local training time (Batch Size)

s, _, b, p = opt_param(P, D, show_result=False)
p2 = p * L
print(s, b, p, p2)

3 97 0.08615903990710687 0.689272319256855


### 2. Experiment Setting

In [3]:
#MNIST
# x_train, y_train, input_size = MNIST_data()
# task = 'clf'

#Room Occupancy
# x_train, y_train, input_size = Room_data()
# task = 'clf'

#Air
# x_train, y_train, input_size = Air_data()
# task = 'reg'

#Conductivity
# x_train, y_train, input_size = Cond_data()
# task = 'reg'

#TOM
x_train, y_train, input_size = Energy_data() #Tom_data()
task = 'reg'

print(input_size, len(y_train))

25 18604


In [4]:
Model_list = []

Model_list.append(OFL_Model('FedOGD', task, K, [False, 0, 0], 1, 1, input_size))
Model_list.append(OFL_Model('OFedAvg', task, K, [False, 0, 0], P, 1, input_size))
Model_list.append(OFL_Model('FedOMD', task, K, [False, 0, 0], 1, int(1/P), input_size))
Model_list.append(OFL_Model('OFedIQ', task, K, [True, s, b], p, 1, input_size))
#Model_list.append(OFL_Model('OFedIQ(L=8)', task, K, [True, s, b], p2, L, input_size))


2023-01-06 02:36:14.784605: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 3. Training

In [7]:
i_max = len(y_train) // K
iter_max = 20
print(iter_max, i_max)

#iter = 15,60 / 80,4 / 40, 9

20 18


In [8]:
for model in Model_list:
    for iter in range(iter_max):
        for i in range(i_max):
            model.train(x_train[K*i : K*(i+1)], y_train[K*i : K*(i+1)], ((i_max * iter) + (i+1)) % model.L)
        print(iter, end =',')
    print()

result_list = []
for model in Model_list:
    result_list.append(model.pull_result())

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,


### 3. Save Result

In [10]:
import pickle

#Code: data / data_90
code = input("Code: ")

for i, model in enumerate(Model_list):
    name = model.name
    with open(f"./result/{task}_{name}_{code}.pkl","wb") as f:
        pickle.dump(result_list[i], f)

### 4. Plot Result

In [9]:
import pickle
import matplotlib.pyplot  as plt

code = input('Code:')

with open(f"./result/{task}_FedOGD_{code}.pkl", "rb") as f:
    r1 = pickle.load(f)
with open(f"./result/{task}_OFedAvg_{code}.pkl", "rb") as f:
    r2 = pickle.load(f)
with open(f"./result/{task}_FedOMD_{code}.pkl", "rb") as f:
    r3 = pickle.load(f)
with open(f"./result/{task}_OFedIQ_{code}.pkl", "rb") as f:
    r4 = pickle.load(f)

l1 = list(range(len(r1))) 
l2 = list(range(len(r2)))
l3 = list(range(len(r3)))
l4 = list(range(len(r4)))

plt.plot(l1, r1, 'black', label=r'FedOGD')
plt.plot(l2, r2, 'red', label=r'OFedAvg ($\mathdefault{p=0.1}$)')
plt.plot(l3, r3, 'blue', label=r'FedOMD ($\mathdefault{L=10}$)')
plt.plot(l4, r4, 'green', label=r'OFedIQ ($\mathdefault{p=0.5151}$)')

plt.xlabel('time step (t)')

if task == 'clf':
    plt.ylabel('Accuracy (t)')
elif task == 'reg':
    plt.ylabel('MSE (t)')
    #plt.yscale("log")

plt.rcParams["font.family"] = "Times New Roman"
plt.grid()
plt.legend()

plt.savefig(f'./Figures/{task}_{code}.png', dpi=200, facecolor="white")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: './result/reg_FedOGD_Energy.pkl'

### 5. $\sigma_{\rm diff}$

In [16]:
x_train, y_train = MNIST_data()
sigma = sigma_diff(Model_list[0], x_train, y_train, 60)
print(sigma)

12.626342124166777


FedOGD w/ iid data: 7.069452932191554

FedOGD w/ non-iid data: 12.626342124166777